# Inicialização

## Carregando bibliotecas e funções de auxílio

In [28]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import xml.etree.ElementTree as et

def parse_xml_to_df(xml_root):
    # Cria um dataframe vazio em que as linhas serão concatenadas.
    df = pd.DataFrame(columns=['similarity', 't', 'h'])
    # Para cada par na root.
    for pair in xml_root:
        # Recupera o valor de t.
        t = pair[0].text
        # Recupera o valor de h.
        h = pair[1].text
        # Recupera o valor da variável target.
        is_entailment = pair.attrib['entailment'] == 'Entailment'
        # Recupera o valor de similaridade atribuído.
        similarity = float(pair.attrib['similarity'])
        # Constroi a nova linha.
        new_line = pd.DataFrame([{'t': t, 'h': h, 'similarity': similarity, 'is_entailment': is_entailment}])
        # Adiciona ao dataframe.
        df = pd.concat([df, new_line], ignore_index=True)
    return df

def get_classification_results(y_train, y_train_pred, y_dev, y_dev_pred, y_test, y_test_pred):
    # Cria um dataframe com todos o resultado de todas as métricas.
    return pd.DataFrame({
        'type':['train', 'dev', 'test'],
        
        'f1': [f1_score(y_train, y_train_pred), 
               f1_score(y_dev, y_dev_pred), 
               f1_score(y_test, y_test_pred)],

        'precision': [precision_score(y_train, y_train_pred), 
                      precision_score(y_dev, y_dev_pred), 
                      precision_score(y_test, y_test_pred)],

        'recall': [recall_score(y_train, y_train_pred), 
                      recall_score(y_dev, y_dev_pred), 
                      recall_score(y_test, y_test_pred)],

        'accuracy': [accuracy_score(y_train, y_train_pred), 
                     accuracy_score(y_dev, y_dev_pred), 
                     accuracy_score(y_test, y_test_pred)],
    })

## Carregando conjuntos de dados

In [4]:
# Recupera o arquivo de entrada de treinamento.
train_xml_root = et.parse('../data/assin2-train.xml').getroot()
# Cria o dataframe de treinamento.
df_train = parse_xml_to_df(train_xml_root)

# Recupera o arquivo de entrada de validação.
dev_xml_root = et.parse('../data/assin2-dev.xml').getroot()
# Cria o dataframe de validação.
df_dev = parse_xml_to_df(dev_xml_root)

# Recupera o arquivo de entrada de validação.
test_xml_root = et.parse('../data/assin2-test.xml').getroot()
df_test = parse_xml_to_df(test_xml_root)



In [9]:
print('Shape de treinamento:', df_train.shape)
print('Shape de validação:', df_dev.shape)
print('Shape de teste:', df_test.shape)

Shape de treinamento: (6500, 4)
Shape de validação: (500, 4)
Shape de teste: (2448, 4)


## Limpando os dados

In [5]:
import unidecode 

def clean_string(x):
    return unidecode.unidecode(x.lower())

# Removendo acentos para normalizar as palavras no conjunto de treinameto.
df_train['t'] = df_train['t'].apply(clean_string)
df_train['h'] = df_train['h'].apply(clean_string)

# Removendo acentos para normalizar as palavras no conjunto de validação.
df_dev['t'] = df_dev['t'].apply(clean_string)
df_dev['h'] = df_dev['h'].apply(clean_string)

# Removendo acentos para normalizar as palavras no conjunto de teste.
df_test['t'] = df_test['t'].apply(clean_string)
df_test['h'] = df_test['h'].apply(clean_string)




# Aplicando Abordagem Simbólica

In [47]:
# A abordagem simbólica utilizada leva em consideração critérios teóricos para a diferenciação entre pares de sentenças com e sem inferências
# Um conjunto de regras é formado para definir se a sentença A acarreta a sentença B
# Considerando que sentenças específicas como "Um cachorro anda rapidamente pela areia da praia" podem acarretar sentenças mais gerais como "Um cachorro anda pela praia", mas não o contrário, considera-se que a sentença A deve ser maior ou igual à B
# Regra 1 de acarretamento: A sentença B deve ser menor ou igual à sentença A em tamanho para que B seja acarretada por A
# Regra 2 de acarretamento: As sentenças devem ser significativamente parecidas - verifica-se se o número de palavras em comum dividido pelo tamanho da sentença B é maior que 50%
# Se ambas as regras forem verdade, a sentença B é classificada como acarretada pela A
# Por fim, se a sentença A contiver o termo "não" e a sentença B não, considera-se que não há relação de inferência entre as sentenças
def symbolic_approach(df_data):
    predictions = []

    for index in df_data.index:   
        sentence_A = df_data['t'][index].split()
        sentence_B = df_data['h'][index].split()
        common_words = [word for word in sentence_A if word in sentence_B]
        
        if len(sentence_B) <= len(sentence_A) and len(common_words)/len(sentence_B) > 0.5:
            prediction = True
        else:
            prediction = False
        if "nao" in sentence_A and "nao" not in sentence_B:
            prediction = False

        predictions.append(prediction) 

    return predictions

y_train_pred = symbolic_approach(df_train)
y_dev_pred = symbolic_approach(df_dev)
y_test_pred = symbolic_approach(df_test)



# Calculando resultados

In [48]:
# Transforma os dados da anotação no formato de vetor binário (0 para não acarretamento, 1 para acarretamento)
y_train = df_train['is_entailment'].astype(int)
y_dev = df_dev['is_entailment'].astype(int)
y_test = df_test['is_entailment'].astype(int)

y_train_pred = np.array(y_train_pred).astype(int)
y_dev_pred = np.array(y_dev_pred).astype(int)
y_test_pred = np.array(y_test_pred).astype(int)

# Calcula as métricas de performance dos resultados do modelo.
df_symbolic_approach_results = get_classification_results(
    y_train, 
    y_train_pred, 
    y_dev, 
    y_dev_pred, 
    y_test, 
    y_test_pred)
display(df_symbolic_approach_results)

,type,f1,precision,recall,accuracy
0,train,0.695185,0.713776,0.677538,0.702923
1,dev,0.716904,0.730290,0.704000,0.722000
2,test,0.685978,0.705527,0.667484,0.694444
